# dim_headquarter

### Importación de librerías

In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:
headquarter = pd.read_sql_table('sede', engineSource)
headquarter.drop(columns=["nombre_contacto", "ciudad_id", "cliente_id"], inplace=True)
headquarter.rename(columns={
  "sede_id": "key_dim_headquarter", 
  "nombre": "name", 
  "direccion": "address",
  "telefono": "phone"
  }, inplace=True)

headquarter

In [ ]:
from datetime import date
headquarter['saved'] = date.today()

### Carga de datos

In [ ]:
headquarter.to_sql("dim_headquarter", engineDestination, index=False, if_exists="replace")